In [1]:
import os
import sys
path1='F:/trec-RTS/25617/bm25_list'
#path1='F:/TREC-RTS/25617/0.1JM_Rank'

#path1='F:/TREC-RTS/25617/1000Dirichlet'


path='C:/Users/lenovo/Google Drive/PHD share folder/paper/trec experiment/analysis'
os.chdir(path)
ip1=['MB226','MB229','MB230','MB239','MB254','MB256','MB258','MB265','MB267','MB276','MB286','MB319','MB320','MB332','MB351']
ip2=['MB358','MB361','MB362','MB363','MB365','MB371','MB377','MB381','MB382','MB391','MB392','MB409','MB410','MB414','MB419']  
ip3=['MB420','MB425','MB431','MB436','MB438','MB440','RTS1','RTS10','RTS13','RTS14','RTS19','RTS2','RTS21','RTS24','RTS25']
ip4=['RTS27','RTS28','RTS31','RTS32','RTS35','RTS36','RTS37','RTS4','RTS43','RTS5','RTS6']
ip=['MB226','MB229','MB230','MB239','MB254','MB256','MB258','MB265','MB267','MB276','MB286','MB319','MB320','MB332','MB351','MB358','MB361','MB362','MB363','MB365','MB371','MB377','MB381','MB382','MB391','MB392','MB409','MB410','MB414','MB419','MB420','MB425','MB431','MB436','MB438','MB440','RTS1','RTS10','RTS13','RTS14','RTS19','RTS2','RTS21','RTS24','RTS25','RTS27','RTS28','RTS31','RTS32','RTS35','RTS36','RTS37','RTS4','RTS43','RTS5','RTS6']
#ip=['MB229']


In [2]:
import pandas as pd
import numpy as np


qrels = pd.read_csv('qrels.txt', sep=' ')
#runfile = pd.read_csv('merge_B_Jelinek0.2_0.6_normal_0.4_21_15_final_luchen.txt', sep=' ')
epoch = pd.read_csv('rts2016-batch-tweets2dayepoch.txt', sep=' ')
cluster = pd.read_csv('clusters.txt', sep='\t')

#print qrels.head() #print runfile.head() print cluster.head() print epoch.head()
cluster.drop(cluster[[3]],axis=1,inplace=True)
cluster.drop(cluster[[3]],axis=1,inplace=True)
cluster.drop(cluster[[3]],axis=1,inplace=True)
#cluster.info() epoch.info()
#print "number of unique:",epoch['tweet_id'].nunique()
#print "number of unique:",epoch['tweet_id','date'].nunique()
#print "number of duplicated:",epoch[epoch.duplicated(['tweet_id','date'])].count()
epoch=epoch.drop_duplicates(subset=['tweet_id','date'])
fqrels = pd.read_csv('Qrel_formated.txt', sep=' ',header=None)
fqrels.columns=['tweet_id','profile','text']
fqrels=fqrels.drop_duplicates(subset=['tweet_id'])


In [4]:
qn=qrels[(qrels['relevance'] > 0)]
qn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3339 entries, 0 to 67507
Data columns (total 4 columns):
profile      3339 non-null object
q0           3339 non-null object
tweet_id     3339 non-null int64
relevance    3339 non-null int64
dtypes: int64(2), object(2)
memory usage: 130.4+ KB


In [ ]:
os.chdir(path1)

for i,p in enumerate(ip):
    qn=qrels[(qrels['relevance'] > 0) & (qrels['profile'] == p)]
    #qn.count()


    ij=qn.merge(epoch,how='inner',on=['tweet_id'])
    #ij.count()
    #ij.head()
#ijc=ij.merge(cluster,how='inner',on=['tweet_id','profile'])
    ijc= pd.merge(ij,cluster,how='inner',on=['tweet_id','profile'])
    #ijc.count()
    ijc=ijc.drop(ijc[[5]],axis=1)
    ijc=ijc.drop(ijc[[1]],axis=1)
    #ijc.info()
    tqrel = ijc.merge(fqrels,how='inner',on=['tweet_id'])
    tqrel=tqrel.sort_values(by=['date'],ascending=[True])
    
    os.chdir(path1)
#os.mkdir(p)
    path2=path1 +"/" + p
    os.chdir(path2)

    fn = p + '_qrel.txt'
    ijc=ijc.sort_values(ijc.columns[3],ascending=[True])
    ijc.to_csv(fn,index=False,sep=' ')
    textstring=  p +'text.txt'
    tqrel.to_csv(textstring,index=False,sep= ' ')
    
    fnstr= p + '_qrel.txt'
    fnstr1 = p + '.txt'
    gq = pd.read_csv(fnstr, sep=' ')
#gq.info()
    pt=pd.read_csv(fnstr1,sep=',',header=None)
    pt.columns=['date','profile','qo','tweet_id','rank','score','text']
#pt.info()
#pt[pt.duplicated(['tweet_id'])]
#gq.drop(gq[[0]],axis=1,inplace=True)
   # pt.drop(pt[[6]],axis=1,inplace=True)
    pt.drop(pt[[2]],axis=1,inplace=True)
    pt1=pt.drop_duplicates(subset='tweet_id')
    pt1.reset_index(drop=True)
    mt=gq.merge(pt1,how='inner',on=['tweet_id','profile','date'])
#mt.info()
#mt.head()
#mt=mt.sort_values(mt.columns[3],mt.columns[5],ascending=[True,True])
    mt=mt.sort_values(by=['date','rank'],ascending=True)
    mt.to_csv(p +'_qrelinrank.txt',index=False,sep=' ')
    pt11=pt1['tweet_id'].tolist()
    gq1=gq['tweet_id'].tolist()

    ngq=list(set(gq1)-set(pt11))
    if len(ngq) > 0 :
        ngq_df = pd.DataFrame(ngq)
        ngq_df.columns=['tweet_id']
        qrelmissRank_df= pd.merge(ngq_df,tqrel,how='inner',on=['tweet_id'])
#qrelmissRank_df= pd.merge(qrelmissRank_df,epoch,how='inner',on=['tweet_id'])
        #nqrelmissRank_df= pd.merge(ngq_df,ijc,how='inner',on=['tweet_id'])
        qrelmissRank_df.to_csv(p + '_qrelmissrank.txt',index=False,sep= ' ')
    #path_rn='C:/Users/lenovo/Google Drive/PHD share folder/paper/trec experiment/rishabh/11417'

    #os.chdir(path_rn)
    os.chdir(path1)
    #run_df=pd.read_csv("merge_B_Dirichlet_0.6_normal_NER_0.4_1000.0 6.0_3.0_+4+3final_doc_l_grid_ndcg1.txt",sep=' ',header=None)

    #run_df=pd.read_csv("merge_B_Jelinek_0.6_normal_NER_0.4_0.1 6.0_3.0_+4+3final_doc_l_grid_ndcg1_again.txt",sep=' ',header=None)
    run_df=pd.read_csv("merge_B_BM25_0.6_normal_NER_0.4_0.1 12_10_+4+3final_doc_l_grid_ndcg1.txt",sep=' ',header=None)
#run_df.drop(run_df[[7]],axis=1,inplace=True)
    run_df.drop(run_df[[6]],axis=1,inplace=True)

    run_df.columns=['date','profile','q0','tweet_id','rank','score']
    run_df.drop(run_df[[2]],axis=1,inplace=True)


    run_df=run_df[(run_df['profile'] == p)]
#run_df.info()
    run_merge_df =pd.merge(mt,run_df,how='inner',on=['tweet_id','profile','date'])

#run_merge_df.head()


    os.chdir(path1)
    os.chdir(p)
    run_merge_df=run_merge_df.sort_values(by=['date','rank_y'],ascending=[True,True])
    run_merge_df.to_csv(p+"_System_summary.txt",index=False,sep=' ')

#run_df.to_csv(p+"_System_summary.csv",index=False)
#Qrel_insumm_df =  pd.DataFrame(list(set(mt['tweet_id'].tolist()) & set(run_df['tweet_id'].tolist())))
#Qrel_insumm_df.head()
    Qrel_notinsumm_df =  pd.DataFrame(list(set(mt['tweet_id'].tolist()) - set(run_df['tweet_id'].tolist())))
    if(Qrel_notinsumm_df.empty==False):
        Qrel_notinsumm_df.columns=["tweet_id"]
        Qrel_notinsumm_df = pd.merge(Qrel_notinsumm_df,mt,how='inner',on=['tweet_id'])
        Qrel_notinsumm_df.info()
#run_merge_df.info()
        Qrel_notinsumm_df=Qrel_notinsumm_df.sort_values(by=['date','rank'],ascending=[True,True])
        Qrel_notinsumm_df.to_csv(p +'System_notincluded.txt',index=False,sep=' ')
    
    #if not (wt_summ_df.empty):
    wt_summ_df =  pd.DataFrame(list(set(run_df['tweet_id'].tolist()) - set(mt['tweet_id'].tolist())))
    #wt_summ_df.info()
    if(wt_summ_df.empty==False):
        wt_summ_df.columns=["tweet_id"]
        wt_summ_df = pd.merge(wt_summ_df,pt1,how='inner',on='tweet_id')
    #wt_summ_df.head()
#wt_summ_df.info()
        wt_summ_df=wt_summ_df.sort_values(by=['date','rank'],ascending=[True,True])
        wt_summ_df.to_csv(p + "NonRelevantTweetinsummary.txt",index=False,sep= ' ')
        
    os.chdir(path1)
    if(run_merge_df.empty==False):
        a=run_merge_df['cluster'].nunique()
    else:
        a=0
    b=len(run_merge_df)
    
    cls = cluster[(cluster['profile'] == p)]
    if(cls.empty==False):
        c= cls['cluster'].nunique()
    else:
        c=0
    if(wt_summ_df.empty==False):
        nrt = len(wt_summ_df)
    else:
        nrt=0
    #qn=qrels[(qrels['relevance'] > 0)
    #print a, b, c
    rt = b-a
    statstr = p + " " + str(c) + " " + str(a) + " " + str (rt) + " " + str(nrt)+"\n"
    #os.chdir(os.pardir)
    fstat = open('stats.txt',"a+")
    fstat.write(statstr)
    fstat.close()
    
    daydf=ijc
    daydf=daydf.drop_duplicates(subset=['date'])
    #daydf=daydf.drop_duplicates(subset=['cluster'])

    summdf = run_df
    summdf=summdf.drop_duplicates(subset=['date'])
    sat=(daydf['date']).tolist()
    sdt=(summdf['date']).tolist()
    gs=[20160802,20160803,20160804,20160805,20160806,20160807,20160808,20160809,20160810,20160811]
    sday = list(set(gs) - set(sat))
    xr = list(set(gs) - set(sdt))
    #x = set(sdt) &  set(sday)
    xssd = len(list(set(gs) - set(sdt)))
    xgssd = len(list(set(gs) - set(sat)))
    wsd = len(list(set(xr)-set(sday)))
    fsd = len(list(set(sday)-set(xr)))

    sdstr = p + " " +str(xgssd) + " " + str(xssd) + " " +str(wsd) +" " + str(fsd)+"\n"
    foutsd = open("silentday.txt",'a+')
    foutsd.write(sdstr)
    foutsd.close()


In [7]:
fqrels.head()

,tweet_id,profile,text
0,763705348339314689,MB267,"RT is reporting:Ukrainian, president, orders, ..."
1,763705444795879424,MB371,Inside the startup that could help GM beat #Go...
2,763707390957121536,RTS37,"RT, 0t_p0ppy: This Male beluga was dead at 15 ..."
3,763708431165456384,MB267,"RT, : BREAKING: Ukrainian president orders for..."
4,763709475530440704,MB267,"RT, : 1/5 US government has seen nothing so fa..."


In [5]:
gs=[20160802,20160803,20160804,20160805,20160806,20160807,20160808,20160809,20160810,20160811]


In [6]:
os.chdir(path1)
daydf=ijc
daydf=daydf.drop_duplicates(subset=['date','cluster'])
daydf=daydf.drop_duplicates(subset=['cluster'])

summdf = run_df
summdf=summdf.drop_duplicates(subset=['date'])
sat=(daydf['date']).tolist()
sdt=(summdf['date']).tolist()
gs=[20160802,20160803,20160804,20160805,20160806,20160807,20160808,20160809,20160810,20160811]
sday = list(set(gs) - set(sat))
xr = list(set(gs) - set(sdt))
#x = set(sdt) &  set(sday)
xssd = len(list(set(gs) - set(sdt)))
xgssd = len(list(set(gs) - set(sat)))
wsd = len(list(set(xr)-set(sday)))
fsd = len(list(set(sday)-set(xr)))

sdstr = p + " " +str(xgssd) + " " + str(xssd) + " " +str(wsd) +" " + str(fsd)+"\n"
foutsd = open("silentday.txt",'a+')
foutsd.write(sdstr)
foutsd.close()

In [44]:
summdf
daydf

,profile,tweet_id,relevance,date,cluster
0,MB226,761356311606984705,1,20160805,1
2,MB226,763555729126993921,1,20160811,2


In [7]:
os.chdir(path1)
a=run_merge_df['cluster'].nunique()
b=len(run_merge_df)
cls = cluster[(cluster['profile'] == p)]

c= cls['cluster'].nunique()
nrt = len(wt_summ_df)
#qn=qrels[(qrels['relevance'] > 0)
print a, b, c
rt = b-a
statstr = p + " " + str(c) + " " + str(a) + " " + str (rt) + " " + str(nrt)+"\n"
#os.chdir(os.pardir)
#fstat = open('stats.txt',"a+")
#fstat.write(statstr)
#fstat.close()

12 22 16


In [26]:
run_merge_df.head()

,profile,tweet_id,relevance,date,cluster,rank_x,score_x,text,rank_y,score_y


In [6]:
pd.DataFrame(list(set(mt['tweet_id'].tolist()) - set(run_df['tweet_id'].tolist())))

""


In [42]:
os.chdir(path1)


In [43]:
os.getcwd()

'F:\\TREC-RTS\\25617\\0.1JM_Rank'